In [2]:
import numpy as np
import math
from scipy.stats import poisson, norm, bernoulli, gamma
from scipy.special import psi

nneuron = 41
sprefs = np.linspace(-60, 60, nneuron)

sig0_sq      = float(3**2)
sig1_sq      = float(12**2)
sigtc_sq     = float(10**2)
ndatapergain = 3000
mu = 0

tau_0 = 1/sig0_sq
tau_1 = 1/sig1_sq
d_tau = tau_0 - tau_1
sig_tc = 10
min_angle = -60
max_angle = 60
step = 3
prefact = np.sqrt(tau_0/tau_1)

def generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, gain, noise):
    s = np.random.normal(mu, np.sqrt(sig0_sq), ndatapergain)
    sprefs_data = np.tile(sprefs, (ndatapergain, 1))
    r = gain * np.exp(-np.square((np.transpose(np.tile(s, (nneuron, 1))) - sprefs_data))/(2 * sigtc_sq))
    if noise == "poisson":
        r = np.random.poisson(r) + 0.0
    return r

In [13]:
#simple version
r = generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, 1, "poisson")
def norm_g(zTw):
    return zTw + 3

def relu(zTw):
    return max(0.000000000000001, zTw)

def g_norm(z, loc, scale):
    d_loc = (z - loc)/scale
    d_scale = -(1/(2 * scale)) + ((z - loc)**2)/(2 * scale**2)
    return [d_loc, d_scale]

L = 1
s = 1
z = [None]*L
W = [None]*L
p_z = [None]*(L + 1)
q_z = [None]*L
p_W = [None]*L
q_W = [None]*L
g = [None]*L
g_W = [None]*L

xi_normal = np.random.rand(nneuron, 2) #[mu, sigma]
train_data = [r[1]]
rho = .001

num = 1
for n in train_data:
    converged = False
    lam_normal = np.random.rand(1, 2)[0] * 3 #[mu, sigma]
    while not converged:
        #normal
        z_loc, z_scale = lam_normal
        z[0] = np.random.normal(loc=z_loc, scale=z_scale, size=s)
        W[0] = [None]*len(xi_normal)
        for i in range(len(xi_normal)):
            W_loc, W_scale = xi_normal[i]
            W[0][i] = np.random.normal(loc=W_loc, scale=W_scale, size=s)
        np.concatenate(W[0])
        p_z[0] = [None] * len(W[0])
        q_z[0] = [None] * len(z[0])
        g[0] = [None] * len(z[0])
        g_W[0] = [None] * len(W[0])
        params = z[0] * W[0]
        for i in range(len(n)):
            lam_p = relu(params[i][0])
            p_z[0][i] = poisson.pmf(n[i], lam_p)
            g_W[0][i] = g_norm(W[0][i], W_loc, W_scale)
        np.concatenate(g_W[0])
        for i in range(len(z[0])):
            q_z[0][i] = norm.pdf(x=z[0][i], loc=z_loc, scale=z_scale)
            g[0][i] = g_norm(z[0][i], z_loc, z_scale)
        p_z[0] = np.log(p_z[0])
        q_z[0] = np.log(q_z[0])
        
        p_z[1] = [norm.pdf(x=z[0][i], loc=0, scale=1)]
        
        l = 0
        S = [None] * L
        T = [None] * L
        for l in range(L):
            S[l] = [None] * len(z[l])
            T[l] = [None] * len(W[l])
            pzl = np.sum(p_z[l])
            for k in range(len(z[l])):
                print g[l][k]
                glk = np.asarray(g[l][k])
                print pzl
                print q_z[l][k]
                S[l][k] = glk * (pzl + p_z[l+1][k] + q_z[l][k]) #???
            for j in range(len(W[l])):
                gWlj = np.asarray(g_W[l][j])
                T[l][j] = gWlj * pzl #???
                np.concatenate(T[l][j])
        lam_normal += list(rho * S[0][0])
        lam_normal[1] = relu(lam_normal[1])
        for m in range(len(xi_normal)):
            rT0n = rho * T[0][m]
            rT0n = [rT0n[0][0], rT0n[1][0]]
            xi_normal[m] += rT0n
            xi_normal[m][1] = relu(xi_normal[m][1])
        num += 1
        if num > 1000:
            converged = True

[-1.1824232027031336, 0.50027886271906985]
-360.479678592
-2.54039289331
[0.24448660731001851, -0.18436289515340937]
-127.71856553
-1.79629111152
[0.66169878220641087, 0.0068421231407115035]
-122.319227919
-1.99550327557
[-0.51278441613003123, -0.080683127578700997]
-235.525509223
-1.90769372838
[2.2179040317268619, 2.2490999368552456]
-234.079851767
-4.24385143668
[-1.0003176014561836, 0.2285573036505511]
-400.639609204
-2.02894367824
[0.40574611707839969, -0.2037443054927808]
-93.4353857669
-1.55966259085
[0.23198236443862461, -0.25601974874644473]
-140.428183104
-1.51526330275
[-0.95496868524063039, 0.17875261474098708]
-528.76441191
-1.96468181154
[-2.0810019376689106, 1.8726737485913545]
-372.82699183
-3.61998782126
[-1.7845852605264803, 1.0943514815848636]
-81.449576982
-2.51527707558
[-1.9508799251076616, 1.3549531290619377]
-93.1572324733
-2.73021365899
[0.22247566960308179, -0.61399920415626807]
-278.42072588
-0.698786030179
[0.31661929759911633, -0.4738971371290282]
-252.9750

/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:55: RuntimeWarning: divide by zero encountered in log
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: RuntimeWarning: invalid value encountered in divide
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: RuntimeWarning: invalid value encountered in divide
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/scipy/stats/_discrete_distns.py:426: RuntimeWarning: invalid value encountered in multiply
  Pk = k*log(mu)-gamln(k+1) - mu
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: RuntimeWarning: invalid value encountered in subtract
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: RuntimeWarning: invalid value encountered in subtract
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/scipy/stats/_discrete_distns.py:426: RuntimeWarning: invalid value encountered in subtra